# Helper functions with Pyodbc

This is mostly developed around connecting to Azure SQL Server databases as that is what I spend most of my SQL time on. 

Only two really big features at this time:
* Connecting from an all purpose keyfile.json instead of building your own connection string. 
* Updating records by simply sending a dataframe. 

I'm hoping to add a lot more to it in the future. 

In [31]:
#libraries that I'm using
import os
import sys
import yaml
import pyodbc
import pandas as pd
import numpy as np

UNAME = os.getlogin()
myPath = os.path.join("C:\\","Users","williamh","source","repos","pyodbc-helper")
sys.path.append(
        os.path.join(myPath)
        )

import odbchelper as sql

First I've made a an expanded connector ```connect_from_dict``` that has defaults for server name, port name, and some security. You can specify these things, but if you didn't you can still connect. 

Defaults:
```
'driver','{ODBC Driver 13 for SQL Server}'
'Server','tcp'
'port',1433
'MultipleActiveResultSets',False
'MultipleActiveResultSets',False
'Timeout',30
'Encrypt',"yes"
'TrustServerCertificate',"no"
```

You also don't have to build a custom connection string. You can use the same keys in your R or node.js applications and build them into one. It also takes care of some synonomous names like ```Name``` and ```Host```.

TODO: 
* Add password prompt (when omitted from keyfile)

In [32]:
#the connection object is very similar in format. 
cnxn = sql.connect_from_dict(os.path.join(myPath,"config_file.json"))
cursor = cnxn.cursor()

## Querying my Uber dataset
I've been collecting my Uber reciepts in my own SQL Database for a while now. 

In [33]:
mytable = "[taxi]"

DF = pd.read_sql_query(f"select top 10 * from {mytable}", cnxn)
DF

,Date of pull,Unique mail id,type,Start Time,End Time,Total Time,Trip Start,Trip End,Amount,emailDate
0,2017-08-21 01:27:54,15769de820be2b37,uberPOOL,21:12:00,21:18:00,00:06:28,"1464-1498 5th Ave, Seattle, WA","417-499 E Roy St, Seattle, WA",4.00,2016-09-27 04:19:31
1,2017-08-23 13:28:29,15774291ef07cf9e,UberX,21:11:00,21:15:00,00:04:44,"1501 Belmont Ave, Seattle, WA","538 Broadway E, Seattle, WA",5.21,2016-09-29 04:17:12
2,2017-09-07 07:28:31,15778614047f32ba,UberX,16:32:00,16:55:00,00:23:52,"21930 SE 51st St, Issaquah, WA","10220 NE Points Dr, Kirkland, WA",26.82,2016-09-29 23:57:00
3,2018-02-24 08:30:13,15783f05521ae2cd,uberPOOL,22:40:00,22:47:00,00:07:03,"2221 Gilman Dr W, Seattle, WA","215 NW 41st St, Seattle, WA",16.91,2016-10-02 05:49:06
4,2019-02-12 14:28:14,1628e6d844d9ee11,uberPOOL,19:00:00,19:12:00,00:11:41,"10220 NE Points Dr, Kirkland, WA","698 Summit Ave E, Seattle, WA",12.64,2018-04-04 02:12:43
5,2018-08-16 01:31:00,157862010b60a3a9,uberPOOL,None,None,00:18:50,"210 NW 41st St, Seattle, WA","511-519 E Roy St, Seattle, WA",5.79,2016-10-02 16:00:29
6,2018-09-08 13:31:17,1578809fdb4fbd57,UberX,17:36:00,17:54:00,00:17:30,"601 Belmont Ave E, Seattle, WA","215 NW 41st St, Seattle, WA",12.39,2016-10-03 00:55:37
7,2019-02-11 02:26:44,167a0a09959917e7,Other,None,None,None,Failed to find,Failed to find,11.64,2018-12-12 04:15:37
8,2018-12-16 08:31:37,1581b13ef3300430,uberPOOL,None,None,00:19:01,"210 NW 41st St, Seattle, WA","2068-2110 NE Pacific St, Seattle, WA",12.69,2016-10-31 14:09:06
9,2019-02-08 20:28:17,1621a9fc2b7be400,uberPOOL,None,None,00:27:53,"208 NW 41st St, Seattle, WA","1994 Burke-Gilman Trail, Seattle, WA",2.41,2018-03-12 14:31:36


## Pushing records in SQL

I'm making a dummy table to show how the process works

In [34]:
df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
df['Date Modified'] = '3-1-2019'
df['index'] = df.index.tolist()

Normally to send anything from Pandas to SQL Server you have to:
* convert NaNs to None types
* Make sure that the column names are all matched up
* Enter a custom SQL command.

```
The ODBC library requires an expression like this:
INSERT INTO [dbo].[table] (col1, col2, col3, col4, col5, col6)
 
VALUES (?,?,?,?,?,?,?,?,?,?) 
```

### Update the table in SQL with your Pandas Dataframe

Only update the values where your key matches the key in the table, else make a new record. 

The ```update_df``` takes care of the insert statement:
```
MERGE Production.ProductInventory AS target  
USING (SELECT ProductID, SUM(OrderQty) FROM Sales.SalesOrderDetail AS sod  
    JOIN Sales.SalesOrderHeader AS soh  
    ON sod.SalesOrderID = soh.SalesOrderID  
    AND soh.OrderDate = @OrderDate  
    GROUP BY ProductID) AS source (ProductID, OrderQty)  
ON (target.ProductID = source.ProductID)  
WHEN MATCHED AND target.Quantity - source.OrderQty <= 0  
    THEN DELETE  
WHEN MATCHED   
    THEN UPDATE SET target.Quantity = target.Quantity - source.OrderQty,   
                    target.ModifiedDate = GETDATE()  
```
It uses the values in your dataframe so you don't have to worry about building out the whole string yourself. It just needs the key.                     

In [35]:
sql.update_df(df,cnxn,table="DB_helper",matchID='index')

In [36]:
pd.read_sql_query(f"select [Date Modified], count(*) from DB_helper GROUP BY [Date Modified]", cnxn)

,Date Modified,
0,2019-02-25,40
1,2019-03-01,13
